In [3]:
# Dependencies
import pymongo
# import datetime
import pandas as pd
from bson.json_util import dumps

from religion_cleaning1 import relig_dict

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.senate_db

# Declare the collections
senators = db.senators_db
uspop = db.us_db
religion = db.religion_db

In [5]:
sen_path = '../resources/SPopulation.csv'
sen_data = pd.read_csv(sen_path)

us_path = '../resources/CensusAttributesData.csv'
us_data = pd.read_csv(us_path)

religion_path = '../resources/USReligionClean.csv'
relig_data = pd.read_csv(religion_path)

In [7]:

senators.delete_many({})

sen_list = []
for p in sen_data.iterrows():
    row = p[1] # p[0] is the index of the row
        
    sen_row = {
        'st': row['st'],
        'state': row['State'],
        'senator': row['Senator'],
        'party': row['party'],
        'born': row['Born'],
        'assumed_office': row['assumed_office'],
        'term_up': row['assumed_office'],
        'population': row['population'],
        'gender': row['gender'],
        'race': row['race'],
        'religion': row['Religion']
    }
    
    sen_list.append(sen_row)
    senators.insert_one(sen_row)

In [9]:
us_data.columns

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'NAME', 'SEX', 'ORIGIN',
       'RACE', 'AGE', 'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016',
       'POPESTIMATE2017', 'POPESTIMATE2018', 'POPESTIMATE2019'],
      dtype='object')

In [11]:
cleaned_us_data = us_data[['STATE', 'NAME', 'SEX', 'ORIGIN', 'RACE', 'AGE', 'POPESTIMATE2019']]
cleaned_us_data.head()

,STATE,NAME,SEX,ORIGIN,RACE,AGE,POPESTIMATE2019
0,1,Alabama,0,0,1,0,35594
1,1,Alabama,0,0,1,1,36679
2,1,Alabama,0,0,1,2,37904
3,1,Alabama,0,0,1,3,38510
4,1,Alabama,0,0,1,4,38476


In [13]:
#uspop.delete_many({})


us_list = []
for p in cleaned_us_data.iterrows():
    row = p[1] # p[0] is the index of the row

    if p[0] % 10000 == 0:
        print(p[0]/10000)
        
    us_row = {
        'NAME': row['NAME'],
        'STATE': row['STATE'],
        'SEX': row['SEX'],
        'ORIGIN': row['ORIGIN'],
        'RACE': row['RACE'],
        'AGE': row['AGE'],
        'POPESTIMATE2019': row['POPESTIMATE2019']
    }

    
    
    us_list.append(us_row)
    #uspop.insert_one(us_row)

# WHY DOES THIS TAKE SO LONG
# 46 secs without mongo
# 3 minutes with??

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0


In [15]:
wyoming = []
for b in us_list:
    if b['NAME'] == 'Wyoming':
        wyoming.append(b)

In [26]:
religion.delete_many({})


relig_list = []
for p in relig_data.iterrows():
    #print(p)
    row = p[1] # p[0] is the index of the row


    #relig_cols = ['index', 'religion', 'percent', 'christian', 'protestant']
        
    relig_row = {
        'Religion': row['Religion'],
        'Percent': row['Percent'],
        'Christian': row['Christian'],
        'Protestant': row['Protestant']
    }

    
    
    relig_list.append(p)

    religion.insert_one(relig_row)


#print(type(relig_list))
    # should definitely make this into a function sigh

In [21]:
json_sen_data = dumps(list(senators.find()))
 
with open('senators.json', 'w') as file: 
    file.write(json_sen_data) 

In [22]:
json_us_data = dumps(list(uspop.find()))
 
with open('uspop.json', 'w') as file: 
    file.write(json_us_data) 

In [27]:
json_relig_data = dumps(list(religion.find()))
 
with open('religion.json', 'w') as file: 
    file.write(json_relig_data) 